### Model preparation

In [1]:
import torch
import torch.nn as nn
from torchvision.models import densenet121
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

num_classes = 8

model = densenet121()
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

### Dataset preparation

In [2]:
class SpectrogramDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, label_map=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = label_map or self._build_label_map()

    def _build_label_map(self):
        labels = self.annotations['label'].unique()
        return {label: idx for idx, label in enumerate(sorted(labels))}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        filename = self.annotations.iloc[idx, 0]
        subfolder = filename[:3]
        img_name = os.path.join(self.root_dir, subfolder, filename)
        print(img_name)
        image = Image.open(img_name).convert('RGB')
        label_str = self.annotations.iloc[idx, 1]
        label = self.label_map[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("project_data/tracks.csv")
df['filename'] = df['track_id'].apply(lambda x: f"{int(x):06d}.png")
df = df.dropna(subset=['genre'])
labels_df = df[['filename', 'genre']].rename(columns={'genre': 'label'})
train_df, val_df = train_test_split(
    labels_df,
    test_size=0.2,
    stratify=labels_df['label'],
    random_state=42
)
train_df.to_csv("project_data/train_set.csv", index=False)
val_df.to_csv("project_data/val_set.csv", index=False)

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = SpectrogramDataset(
    csv_file='project_data/train_set.csv',
    root_dir='project_data/spectrograms',
    transform=transform
)

val_dataset = SpectrogramDataset(
    csv_file='project_data/val_set.csv',
    root_dir='project_data/spectrograms',
    transform=transform
)

In [12]:
#validation
image, label = train_dataset[1]
print(image.shape)
print(label)
print(train_dataset.label_map)


project_data/spectrograms\006\006330.png
torch.Size([3, 224, 224])
5
{'Electronic': 0, 'Experimental': 1, 'Folk': 2, 'Hip-Hop': 3, 'Instrumental': 4, 'International': 5, 'Pop': 6, 'Rock': 7}


### Model training

In [14]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

NameError: name 'train_loader' is not defined